In [1]:
import getpass
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from pprint import pprint
from pyproj import Transformer
import rasterio
from rasterio.mask import mask
from rasterio.windows import from_bounds
import requests
from shapely.geometry import Polygon, mapping, shape
from requests.auth import HTTPBasicAuth
from tqdm import tqdm

from rasterio.windows import Window
from PIL import Image
def image_download(folder_name, download_links):
    
    #check if folder for downloading exists, if not create
    if not (os.path.isdir(folder_name)):
        print('Figure directory didn''t exist, creating now.')
        os.mkdir(folder_name)
    else:
        print('Figure directory exists.') 
        
        
    for image_name, download_link in tqdm(download_links.items()):
        try:
            # Use allow_redirects=False to prevent automatic redirects
            response = requests.get(download_link, stream=True, allow_redirects=True)

            if response.status_code == 302:  # 302 indicates a redirect
                # Get the final URL from the 'Location' header
                final_url = response.headers.get('Location')

                if final_url:
                    response = requests.get(final_url, stream=True)
                else:
                    print(f"Failed to retrieve {image_name}. Redirect URL not found.")
                    continue

            if response.status_code == 200:
                local_file_path = f'./{folder_name}/{image_name}.tif'

                with open(local_file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)

#                 print(f"Downloaded {image_name} as {local_file_path}")
            else:
                print(f"Failed to retrieve {image_name}. Status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred while downloading {image_name}: {e}")
            
#     print(f'Downloading completed at {datetime.datetime.now()}')


In [2]:
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = '1c61ae19230448e19b5bc359cbf1232e'
    
session = requests.Session()
session.auth = (API_KEY, "")

with open('../Documents/Blooms/bloom_geojsons/bridge.geojson', 'r') as file:
    data = json.load(file)

In [76]:
geojson_geometry = {
    "type": "Polygon",
    "coordinates": data['features'][0]['geometry']['coordinates']
}

# images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# only  images which have < 5% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

quality_filter = {
   "type":"StringInFilter",
   "field_name":"quality_category",
   "config":[
      "standard","test"]
}
#  images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2024-01-31T00:00:00.000Z",
    "lt": "2024-06-30T00:00:00.000Z"
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter, quality_filter]
}

item_type = "PSScene"

search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

results = search_result.json()
feat = results['features'][0]
item_geom = feat['geometry']
image_ids = [feature['id'] for feature in results['features']]
print(len(image_ids))

102


In [77]:
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )
# print(result.json().keys())


In [81]:
for image_id in image_ids:
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, image_id)

    try:
        # Returns JSON metadata for assets in this ID.
        result = requests.get(id_url, auth=HTTPBasicAuth(API_KEY, ''))
        links = result.json()[u"ortho_visual"]["_links"] 
        self_link = links["_self"]
        activation_link = links["activate"]

        # Request activation of the 'ortho_analytic_4b' asset:
        activate_result = \
          requests.get(
            activation_link,
            auth=HTTPBasicAuth(API_KEY, '')
          )
        activation_status_result = \
          requests.get(
            self_link,
            auth=HTTPBasicAuth(API_KEY, '')
          )
        print(f'status: {activation_status_result.json()["status"]} for {image_id}')
    #     download_link = activation_status_result.json()["location"]
    except Exception as e:
        print(e)
        pass

status: active for 20240605_151101_55_24ce
status: active for 20240528_160049_99_2488
status: active for 20240402_150710_18_2415
status: active for 20240314_160001_86_24f5
status: active for 20240202_150235_44_24bc
status: active for 20240629_160631_90_247c
status: active for 20240626_160151_86_24bd
status: active for 20240625_150854_25_24b3
status: active for 20240625_152113_91_2465
status: active for 20240622_160032_14_24e5
status: active for 20240620_160607_03_2484
status: active for 20240617_150709_95_24b2
status: active for 20240616_155954_46_24f8
status: active for 20240616_155818_86_24e1
status: active for 20240616_160136_98_24f4
status: active for 20240612_160124_74_24fb
status: active for 20240611_160602_55_2490
status: active for 20240611_160604_41_2490
status: active for 20240609_155941_31_24eb
status: active for 20240609_155558_55_227a
status: active for 20240607_155834_69_24d5
status: active for 20240607_155832_45_24d5
status: active for 20240605_150932_64_24c3
status: act

In [82]:
download_links = {}
for image_id in tqdm(image_ids):
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, image_id)
    try:
        # Returns JSON metadata for assets in this ID.
        result = requests.get(id_url, auth=HTTPBasicAuth(API_KEY, ''))
        asset_info = result.json().get("ortho_visual")  # Replace with the appropriate asset type

        if asset_info:
            links = asset_info["_links"]
            self_link = links["_self"]
            activation_link = links["activate"]

            # Request activation of the asset:
            activate_result = requests.get(activation_link, auth=HTTPBasicAuth(API_KEY, ''))
            activation_status_result = requests.get(self_link, auth=HTTPBasicAuth(API_KEY, ''))

            activation_status = activation_status_result.json()
            download_link = activation_status.get("location")

            if download_link:

                download_links[image_id] = download_link
    #             print(f"Download link for image ID {image_id}: {download_link}")
            else:
                print(f"Activation status for image ID {image_id} is not available.")

        else:
            print(f"Asset 'ortho_visual' not found for image ID {image_id}")
    except Exception as e:
        print(e)
        pass
# Now, download_links dictionary contains download links for each image ID.
# last_half = dict(list(download_links.items())[187:]) 

100%|█████████████████████████████████████████| 102/102 [01:25<00:00,  1.19it/s]


In [83]:
image_path = 'bridge'  

image_download(image_path, download_links)

Figure directory exists.


100%|█████████████████████████████████████████| 102/102 [05:58<00:00,  3.51s/it]


In [85]:
lat, lon = 36.0476, -76.69611

x,y = 300,300
# image_path = 'planet9'  

cropped_img_folder_name = 'crops'
if not (os.path.isdir(image_path+'/'+cropped_img_folder_name)):
        print('Figure directory didn''t exist, creating now.')
        os.mkdir(image_path+'/'+cropped_img_folder_name)
else:
    print('Figure directory exists.') 
       
        
for fi in sorted(os.listdir(image_path)):
    if fi.endswith(".tif"):
        working_path = os.path.join(image_path, fi)
        with rasterio.open(working_path) as rds:
            
            transformer = Transformer.from_crs("EPSG:4326", rds.crs, always_xy=True)
            xx, yy = transformer.transform(lon, lat)
            row, col = rds.index(xx, yy)
            if row > x and col > y:
                try:
                    window = Window.from_slices(rows = (row-x, row+x), cols =(col-y, col+y))
                    data = rds.read(window=window)
                    data = np.moveaxis(data, 0, 2) 
                    img_arr = np.array(data)
                    black_space = np.mean(img_arr[:,:,:]/255)

                    if black_space < 0.25 or black_space >= 0.9 or img_arr.shape[0] != x*2 or img_arr.shape[1] != y*2 or img_arr.shape[2] != 4:
                        print(f'blank_space:{black_space}')
#                         plt.imshow(data)
#                         plt.axis('off')
#                         plt.show()
                    else:
#                         plt.imshow(img_arr[:,:,:3])
#                         plt.title(fi)
#                         plt.axis('off')
#                         plt.show()
                        im = Image.fromarray(data)
                        im.save(image_path+'/'+cropped_img_folder_name+'/'+str(fi))  
                        print(f'Image saved for {fi}')
                        
                except Exception as e:
                    print(e)
                    pass
            else: pass

            if row < x or col <y:
                print('invalid row or col length: ',row, col)
                pass    

Figure directory exists.
Image saved for 20160102_185910_0c18.tif
Image saved for 20160410_193310_0c66.tif
Image saved for 20160708_145903_0e0d.tif
Image saved for 20161015_173446_0c54.tif
Image saved for 20161017_150234_0e0e.tif
Image saved for 20161020_150342_0e0f.tif
blank_space:0.08404141884531591
blank_space:0.06716824074074075
Image saved for 20161020_152233_0c13.tif
Image saved for 20161021_150041_1_0c27.tif
blank_space:0.10796122622671772
Image saved for 20161025_132709_0c46.tif
Image saved for 20161026_150248_0e20.tif
Image saved for 20161106_150313_0e19.tif
blank_space:0.11103452069716777
Image saved for 20161108_152151_0c44.tif
Image saved for 20161110_142200_0c75.tif
Image saved for 20161112_150356_0e26.tif
Image saved for 20161122_150347_0e0e.tif
Image saved for 20161207_200123_1_0c22.tif
Image saved for 20161209_150423_0e2f.tif
Image saved for 20161209_191345_0c42.tif
Image saved for 20161209_193438_0c38.tif
Image saved for 20161209_193439_0c38.tif
Image saved for 2016121

Image saved for 20180420_151728_1040.tif
Image saved for 20180421_151738_103d.tif
Image saved for 20180422_151721_0e0f.tif
Image saved for 20180428_153100_0f33.tif
blank_space:0.2351107077591742
Image saved for 20180501_151816_1018.tif
Image saved for 20180502_151625_0e19.tif
blank_space:0.08294988017429193
Image saved for 20180502_151640_1036.tif
Image saved for 20180503_151809_1013.tif
Image saved for 20180504_151836_0f31.tif
blank_space:0.18121206427015252
Image saved for 20180504_183235_0f06.tif
Image saved for 20180511_151754_101e.tif
blank_space:0.1318264079520697
Image saved for 20180512_151832_103a.tif
Image saved for 20180514_151724_1003.tif
Image saved for 20180524_151845_1027.tif
Image saved for 20180527_151801_1029.tif
Image saved for 20180527_151802_1029.tif
Image saved for 20180601_151629_0e19.tif
Image saved for 20180604_152548_0f4d.tif
blank_space:0.15817050653594772
blank_space:0.12098320261437912
Image saved for 20180607_152014_0f28.tif
invalid row or col length:  195

Image saved for 20190422_152111_0e26.tif
Image saved for 20190422_160116_66_1063.tif
Image saved for 20190423_143646_1020.tif
Image saved for 20190423_152736_0f35.tif
Image saved for 20190425_144010_0f33.tif
Image saved for 20190425_152705_0f25.tif
Image saved for 20190427_152654_100c.tif
Image saved for 20190503_143506_1054.tif
Image saved for 20190507_143703_1053.tif
blank_space:0.1461843300653595
Image saved for 20190508_153049_0f17.tif
Image saved for 20190515_153022_1034.tif
Image saved for 20190518_152705_1001.tif
blank_space:0.22863848311546842
Image saved for 20190519_143305_0f33.tif
blank_space:0.08492462418300656
blank_space:0.1408373175381263
Image saved for 20190520_152308_0e2f.tif
Image saved for 20190520_152750_0f34.tif
Image saved for 20190520_160100_19_106f.tif
Image saved for 20190520_160102_29_106f.tif
Image saved for 20190522_143511_104e.tif
invalid row or col length:  2596 -163
invalid row or col length:  154 389
Image saved for 20190524_143322_0f3c.tif
blank_space:

Image saved for 20200329_153203_0f4e.tif
invalid row or col length:  3130 178
Image saved for 20200402_155339_50_1057.tif
Image saved for 20200403_134139_0f49.tif
invalid row or col length:  56 1456
Image saved for 20200403_155540_80_1066.tif
Image saved for 20200404_134559_0f46.tif
Image saved for 20200405_134545_1052.tif
Image saved for 20200412_134258_1049.tif
Image saved for 20200414_152959_100a.tif
Image saved for 20200417_152924_101f.tif
Image saved for 20200419_154146_92_1063.tif
Image saved for 20200419_155608_32_105d.tif
Image saved for 20200422_153204_1012.tif
Image saved for 20200503_153031_1003.tif
blank_space:0.2172956177763007
Image saved for 20200508_150606_26_2257.tif
Image saved for 20200510_153049_1032.tif
Image saved for 20200511_133911_0f36.tif
Image saved for 20200511_150425_40_2278.tif
Image saved for 20200515_153145_1001.tif
Image saved for 20200516_133837_0f36.tif
blank_space:0.07363173474945535
Image saved for 20200516_150640_61_2304.tif
Image saved for 2020051

Image saved for 20210225_152607_45_1065.tif
blank_space:0.12971909287180577
blank_space:0.21386260076252725
Image saved for 20210304_153203_1035.tif
Image saved for 20210305_155556_44_2254.tif
Image saved for 20210306_155524_39_2426.tif
Image saved for 20210307_151313_35_2276.tif
Image saved for 20210307_155604_48_2412.tif
Image saved for 20210309_124844_1048.tif
Image saved for 20210310_124757_100d.tif
blank_space:0.1787714549587951
Image saved for 20210311_150533_52_2212.tif
Image saved for 20210311_150535_73_2212.tif
Image saved for 20210311_150627_64_2459.tif
Image saved for 20210311_152449_00_1063.tif
blank_space:0.10637842047930285
Image saved for 20210312_150217_03_2449.tif
Image saved for 20210312_150316_88_2420.tif
Image saved for 20210314_153059_1005.tif
blank_space:0.20238879357298478
Image saved for 20210321_124525_1050.tif
Image saved for 20210329_124640_0f3c.tif
Image saved for 20210329_150726_77_2434.tif
Image saved for 20210329_150729_12_2434.tif
Image saved for 2021032

Image saved for 20210913_152806_1012.tif
Image saved for 20210914_150258_51_241d.tif
Image saved for 20210915_145801_70_241b.tif
Image saved for 20210915_150025_0e26.tif
Image saved for 20210916_121707_100d.tif
Image saved for 20210916_121708_100d.tif
Image saved for 20210920_121547_0f21.tif
Image saved for 20210920_150117_40_2432.tif
Image saved for 20210921_151314_34_106a.tif
Image saved for 20210921_152003_99_222f.tif
Image saved for 20210921_154830_02_2403.tif
Image saved for 20210924_150512_37_245c.tif
Image saved for 20210925_152713_1008.tif
Image saved for 20210926_150118_80_242d.tif
Image saved for 20210927_145801_89_2456.tif
Image saved for 20210927_153256_0f4e.tif
blank_space:0.16874063495921782
Image saved for 20210929_150113_71_2458.tif
Image saved for 20210930_152631_101f.tif
Image saved for 20210930_155339_1105.tif
Image saved for 20211001_150305_75_241d.tif
Image saved for 20211001_150308_05_241d.tif
Image saved for 20211001_153103_1002.tif
Image saved for 20211003_16195

Image saved for 20220519_152905_33_249e.tif
Image saved for 20220519_152907_63_249e.tif
Image saved for 20220521_152913_20_247c.tif
Image saved for 20220521_152915_49_247c.tif
Image saved for 20220528_145523_35_241e.tif
Image saved for 20220528_152652_05_249d.tif
Image saved for 20220529_152704_15_2477.tif
Image saved for 20220529_152706_46_2477.tif
Image saved for 20220601_150951_85_2262.tif
Image saved for 20220602_152653_60_2486.tif
Image saved for 20220605_152916_18_2477.tif
Image saved for 20220606_152832_49_2446.tif
Image saved for 20220613_152819_27_2483.tif
Image saved for 20220613_154226_77_2414.tif
Image saved for 20220613_154229_06_2414.tif
Image saved for 20220617_154421_68_2426.tif
Image saved for 20220618_153954_14_2424.tif
Image saved for 20220618_154407_80_2414.tif
Image saved for 20220619_152754_04_2495.tif
Image saved for 20220620_152852_51_2483.tif
Image saved for 20220620_153002_86_2446.tif
blank_space:0.2401421595860567
Image saved for 20220621_152745_68_24a5.tif
b

Image saved for 20230419_152936_73_249c.tif
Image saved for 20230420_153119_75_247f.tif
Image saved for 20230423_144915_62_2447.tif
Image saved for 20230425_153144_21_247b.tif
Image saved for 20230425_153146_37_247b.tif
Image saved for 20230501_153326_31_2473.tif
Image saved for 20230502_145458_35_2427.tif
Image saved for 20230502_145557_54_2430.tif
Image saved for 20230504_152855_36_248e.tif
Image saved for 20230505_145325_41_242b.tif
Image saved for 20230506_145948_60_24b2.tif
Image saved for 20230507_152052_90_2276.tif
Image saved for 20230509_153256_71_24a3.tif
Image saved for 20230510_145405_92_245c.tif
blank_space:0.24939570806100217
Image saved for 20230511_145643_12_242d.tif
Image saved for 20230511_145645_26_242d.tif
Image saved for 20230517_145543_81_245c.tif
Image saved for 20230517_145545_94_245c.tif
Image saved for 20230522_145827_40_24cf.tif
Image saved for 20230522_145953_18_24c5.tif
Image saved for 20230522_153546_26_2478.tif
Image saved for 20230523_145852_86_24bf.tif


Image saved for 20240311_155953_70_24df.tif
Image saved for 20240311_160050_40_24e5.tif
Image saved for 20240312_150831_58_2447.tif
Image saved for 20240312_155517_99_2490.tif
Image saved for 20240312_155519_97_2490.tif
Image saved for 20240314_155959_60_24f5.tif
Image saved for 20240314_160001_86_24f5.tif
Image saved for 20240315_150757_44_24d0.tif
Image saved for 20240315_150759_57_24d0.tif
Image saved for 20240315_151232_55_2440.tif
Image saved for 20240319_150639_64_24c8.tif
Image saved for 20240319_150832_05_24bb.tif
Image saved for 20240321_150634_09_242b.tif
Image saved for 20240321_150635_99_242b.tif
Image saved for 20240325_151043_17_2465.tif
invalid row or col length:  76 1690
invalid row or col length:  4796 223
Image saved for 20240331_155307_92_2488.tif
Image saved for 20240402_150710_18_2415.tif
Image saved for 20240405_155749_21_24e6.tif
Image saved for 20240406_155619_03_2490.tif
Image saved for 20240407_150658_53_24b3.tif
Image saved for 20240407_150700_71_24b3.tif
bla

In [105]:
len(os.listdir('planet7/ablemarle_sound/'))

1334